
# <center>Resection problem in 3D</center>

  \----------------------------------------------------------  
Author: Jorge Ventura Gil  
Date:   October, 2023  
License: BSD-3  
Version: 1.0  
  \----------------------------------------------------------  

The first step is to include the libraries and dependencies to work with GA.

In [2]:
from clifford import *
from clifford.g2c import *
from math import *

### Functions


- **GetAngles**    
This auxiliary function is in charge of obtaining the angles alpha, beta and gamma knowing the position of the points A, B, C and P. 

- **VGAMethod_2D**  
The function applies the VGA-based method for solving the 2D rejection problem.

- **CGA_Cassini_Method**  
The function applies the CGA Cassini method for solving the 2D rejection problem.

- **CGA_Collins_Method**  
The function applies the CGA Collins method for solving the 2D rejection problem.

In [3]:
def GetAngles(A, B, C, P):
    
    ua = [A[0] - P[0], A[1] - P[1]]
    ub = [B[0] - P[0], B[1] - P[1]]
    uc = [C[0] - P[0], C[1] - P[1]]
    
    theta_ua = atan2(ua[1], ua[0])
    theta_ub = atan2(ub[1], ub[0])
    theta_uc = atan2(uc[1], uc[0])
    
    
    print("Angles ----> A: %f, B: %f, C: %f\n"%(theta_ua*180/pi, theta_ub*180/pi, theta_uc*180/pi))
    
    alpha =  theta_ua - theta_ub
    beta =  theta_ub - theta_uc
    
    # If P is aligned with A and B or B and C, a new assignment of the points is necessary.
    if beta == 0:
        alpha =  theta_ub - theta_ua
        beta =  theta_ua - theta_uc     
        print("The angle beta = 0 ---> The position of A and B will be interchanged.")
        A[:], B[:] = B[:], A[:]
        
    if alpha == 0:
        alpha =  theta_ua - theta_uc
        beta =  theta_uc - theta_ub
        
        print("The angle alpha = 0 ---> The position of C and B will be interchanged.")
        C[:], B[:] = B[:], C[:]
    
    # if we are working in 2 dimensions, gamma is not used.
    gamma = None
    if len(A) == 3:
        gamma = atan2(A[2] - P[2], sqrt(ua[0]**2 + ua[1]**2))
        

    return alpha, beta, gamma

#=========================================================================================================

def VGA_Method_2D(A,B,C,alpha,beta):
    
    
    v1 = (A[0]-B[0])*e1 + (A[1]-B[1])*e2
    v2 = (C[0]-B[0])*e1 + (C[1]-B[1])*e2
        
    d1 = v1 + (v1/tan(alpha))*e12
    d2 = v2 - (v2/tan(beta))*e12
    d = d2-d1
    
    try:
        p = (d1^d)|d.inv()
        P = [B[0] + p.as_array()[1], B[1] + p.as_array()[2]]
        
    except Exception as err:
        print("Prohibited circle ---> ",err)
        
    return P

#=========================================================================================================

def CGA_Cassini_Method(A,B,C,alpha,beta):
    
    a_v = A[0]*e1 + A[1]*e2 
    b_v = B[0]*e1 + B[1]*e2
    c_v = C[0]*e1 + C[1]*e2

    a = up(a_v)
    b = up(b_v)
    c = up(c_v)

    LAB = a ^ b ^ einf
    LBC = c ^ b ^ einf

    midAB = (a - b).dual().normal()
    midBC = (b - c).dual().normal()
    
    Ra = e**(-((alpha - pi/2)/2)*e12)
    Rb = e**(-((pi/2 - beta)/2)*e12)
    
    Ta = 1 + einf*(a_v)/2
    Tc = 1 + einf*(c_v)/2
    
    LAO = (Ta*Ra*~Ta)*LAB*~(Ta*Ra*~Ta)
    LCO = (Tc*Rb*~Tc)*LBC*~(Tc*Rb*~Tc)
    
    O1 = up(down(eo << (LAO.dual() ^ midAB.dual()).dual().normal()))
    O2 = up(down(eo << (LCO.dual() ^ midBC.dual()).dual().normal()))
    
    R1 = (sqrt(-2*(O1 << b)))
    R2 = (sqrt(-2*(O2 << b)))

    C1 = (O1 - 1/2*(R1**2)*einf).normal()
    C2 = (O2 - 1/2*(R2**2)*einf).normal()

    P = (C1 ^ C2).dual()
    
    P1 = (sqrt(P << P) + P)/(einf << P)
    P2 = (-sqrt(P << P) + P)/(einf << P)

    
    if down(P1) == down(b):
        return down(P2)

    else:
        return down(P1)
    
#==========================================================================================================
    
def CGA_Collins_Method(A,B,C,alpha,beta):
    
    a_v = A[0]*e1 + A[1]*e2 
    b_v = B[0]*e1 + B[1]*e2
    c_v = C[0]*e1 + C[1]*e2

    a = up(a_v)
    b = up(b_v)
    c = up(c_v)
    
    LAC = a ^ c ^ einf
    
    Rb = e**(-(beta/2)*e12)
    Ra = e**(-(-alpha/2)*e12)
    
    Ta = 1 + einf*(a_v)/2
    Tc = 1 + einf*(c_v)/2
    
    L1 = (Ta*Rb*~Ta)*LAC*~(Ta*Rb*~Ta)
    L2 = (Tc*Ra*~Tc)*LAC*~(Tc*Ra*~Tc)
    
    # L1 and L2 are parallel
    if (einf<<L1) == (einf<<L2):
    
        V = einf << L1
        LEB = b ^ V
        F = (LAC.dual() ^ LEB.dual()).dual()
        P = up(down(eo<<F))
        
        return P
    
    # L1 and L2 intersect
    E = up(down(eo << (L1.dual() ^ L2.dual()).dual()))
    C = a ^ c ^ E
    LEB = E ^ b ^ einf

    P = (C.dual() ^ LEB.dual()).dual().normal()
    
    P1 = (sqrt(P << P) + P)/(einf << P)
    P2 = (-sqrt(P << P) + P)/(einf << P)
    
    if down(P1) == down(E):
        return down(P2)
    else:
        return down(P1)


### Examples

1. Point P outside the triangle formed by A, B and C

In [52]:
A = [-7,-1]
B = [1,5]
C = [15.6,6]
P = [3.12,-18.5]

2. Point P inside the triangle formed by points A, B, and C

In [4]:
A = [-5,4]
B = [3.5,10]
C = [7.6,-1]
P = [3,3.5]

3. Point P aligned with A and C

In [6]:
A = [5,6]
B = [0,1]
C = [3,12]
P = [8,-3]

Choose a set of points and run the following code to check how each method is able to correctly calculate the position of point P

In [7]:
alpha, beta, _ = GetAngles(A,B,C,P)
conv = 180/pi
print("ALPHA: %fº\nBETA: %fº"%(alpha*conv,beta*conv))

print("\n================\nVGA-based Method\n================\n")
Pvga = VGA_Method_2D(A,B,C,alpha,beta)
print("The position of point P is: [%f, %f]"%(Pvga[0],Pvga[1]))

print("\n==================\nCGA-Cassini Method\n==================\n")
Pcass = CGA_Cassini_Method(A,B,C,alpha,beta)
print("The position of point P is: [%f, %f]"%(Pcass.as_array()[1],Pcass.as_array()[2]))

print("\n==================\nCGA-Collins Method\n==================\n")
Pcoll = CGA_Collins_Method(A,B,C,alpha,beta)
print("The position of point P is: [%f, %f]"%(Pcoll.as_array()[1],Pcoll.as_array()[2]))

Angles ----> A: 108.434949, B: 153.434949, C: 108.434949

ALPHA: -45.000000º
BETA: 45.000000º

VGA-based Method

The position of point P is: [8.000000, -3.000000]

CGA-Cassini Method

The position of point P is: [8.000000, -3.000000]

CGA-Collins Method

The position of point P is: [8.000000, -3.000000]


## Solving the problem in 3D

### Function
- **CGA_Method_3D**  
This method is based on using any of the previous functions to solve the problem in 2D. With the 2D solution we obtain the remaining coordinate using the angle _gamma_.

In [13]:
from clifford.g3c import *
from clifford.tools.g3c import *

def CGA_Method_3D(A,B,C,alpha,beta,gamma, P2d):
    
    P_v = P2d[0]*e1 + P2d[1]*e2
    P_c = up(P_v)
    
    a = A[0]*e1 + A[1]*e2 + A[2]*e3
    b = B[0]*e1 + B[1]*e2 + B[2]*e3
    c = C[0]*e1 + C[1]*e2 + C[2]*e3
        
    A = up(a)
    B = up(b)  
    C = up(c)

    L = P_c ^ up(P_v + e3) ^ einf
    Na = ((A << L).dual()) ^ einf
    F = (Na.dual() ^ L.dual()).dual()
    g =  up(down(eo << F))

    P = down(g) + e3 * tan(-gamma) * sqrt(-2* (g << A))

    return P
    

3D point configuration

In [10]:
A = [1,15,0]
B = [-4,1,3]
C = [3,-8,2.6]
P = [6.81034,-6.63073,14.53887809]

In [12]:
alpha, beta, gamma = GetAngles(A,B,C,P)
conv = 180/pi
print("ALPHA: %f º\nBETA: %f º\nGAMMA: %f º"%(alpha*conv,beta*conv,gamma*conv))

p = CGA_Method_3D(A,B,C,alpha,beta,gamma,P)
print("The position of point P is: [%f, %f, %f]"%(p.as_array()[1],p.as_array()[2],p.as_array()[3]))

Angles ----> A: 105.035590, B: 144.782755, C: -160.233776

ALPHA: -39.747164 º
BETA: 305.016531 º
GAMMA: -32.988778 º
The position of point P is: [6.810340, -6.630730, 14.538878]


## Time Measurement

In [14]:
import time
from clifford.g2c import *

A = [-10,-5]
B = [-1,6]
C = [5,-8,]
P = [-12.5,-0.6]

alpha, beta, _ = GetAngles(A,B,C,P)

Pf = CGA_Cassini_Method(A,B,C, alpha, beta)
Pf = CGA_Collins_Method(A,B,C, alpha, beta)
Pf = VGA_Method_2D(A,B,C, alpha, beta)

it = 1000

# Cassini
t_ini = time.time()

for i in range(it):
    CGA_Cassini_Method(A,B,C, alpha, beta)

t_end = time.time()

t_cassini = t_end - t_ini

#Collins
t_ini = time.time()

for i in range(it):
    CGA_Collins_Method(A,B,C, alpha, beta)

t_end = time.time()

t_collins = t_end - t_ini

#VGA
t_ini = time.time()

for i in range(it):
    VGA_Method_2D(A,B,C, alpha, beta)

t_end = time.time()

t_vga = t_end - t_ini

print("Cassini time: %f"%(t_cassini))
print("Collins time: %f"%(t_collins))
print("VGA time: %f"%(t_vga))


Angles ----> A: -60.395549, B: 29.852076, C: -22.921419

Cassini time: 0.534005
Collins time: 0.463000
VGA time: 0.062001
